In [30]:
!pip install -q streamlit


In [3]:
%pwd

'/content'

In [31]:
%%writefile app.py
import streamlit as st
import datetime as dt
from datetime import date
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
#import shutil
from sklearn.preprocessing import MinMaxScaler
#from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt

from darts import TimeSeries
from darts.models import RNNModel, ExponentialSmoothing, BlockRNNModel
from darts.metrics import mape, mse
#from darts.utils.statistics import check_seasonality, plot_acf
#from darts.datasets import AirPassengersDataset, SunspotsDataset
from darts.utils.timeseries_generation import datetime_attribute_timeseries
from darts.models import forecasting
from darts.dataprocessing.transformers import Scaler, MissingValuesFiller
import yfinance as yf


TODAY = dt.date.today().strftime("%Y-%m-%d")

START ="2022-05-06"

st.title('Stock Forecast App')

stocks = ('AAPL', 'ACN', 'ADBE', 'ADI', 'ADSK', 'AKAM', 'AMAT', 'AMD', 'ANET', 'ANSS', 'AVGO', 'CDNS', 'CDW', 'CRM', 'CSCO', 'CTSH', 'DXC', 'ENPH', 'EPAM', 'FFIV', 'FICO', 'FSLR', 'FTNT', 'GLW', 'INTU', 'IT', 'JNPR', 'KEYS', 'KLAC', 'LRCX', 'MCHP', 'MPWR', 'MSI', 'MU', 'NOW', 'NTAP', 'NVDA', 'NXPI', 'ON', 'PTC', 'QCOM', 'QRVO', 'ROP', 'ROP', 'SEDG', 'SNPS', 'SWKS', 'TEL', 'TER', 'TRMB', 'TXN', 'TYL', 'VRSN', 'ZBRA')
selected_stock = st.selectbox('Select the stock you wish to view/predict', stocks)

def load_data(ticker):
    data = yf.download(ticker, START, TODAY)
    data.reset_index(inplace=True)
    return data

def transform_data(ticker):
    aapl= yf.Ticker(ticker)
    df = aapl.history(start="2022-04-01", end="2023-04-01", interval="1d")
    idx = pd.date_range(start='2022-04-01', end='2023-04-01')
    df.index = pd.DatetimeIndex(df.index)
    df.index = df.index.tz_localize(None) 
    df = df.reindex(idx, method = 'pad')
    series = TimeSeries.from_dataframe(df)
    adj_series = series.drop_columns(['Open', 'High', 'Low', 'Volume', 'Dividends', 'Stock Splits'])
    return adj_series

data_load_state = st.text('Loading data...')
data = load_data(selected_stock)
data_load_state.text('Loading data... done!')

transform_data_state = st.text('Transforming data...')
transform_data_stock = transform_data(selected_stock)
transform_data_state.text('Transforming data...done')

st.subheader('Raw data')
st.write(data.tail())
st.subheader('Transformed data')
st.write(transform_data_stock)


Overwriting app.py


In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
#import shutil
from sklearn.preprocessing import MinMaxScaler
#from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt

from darts import TimeSeries
from darts.models import RNNModel, ExponentialSmoothing, BlockRNNModel
from darts.metrics import mape, mse
#from darts.utils.statistics import check_seasonality, plot_acf
#from darts.datasets import AirPassengersDataset, SunspotsDataset
from darts.utils.timeseries_generation import datetime_attribute_timeseries
from darts.models import forecasting
from darts.dataprocessing.transformers import Scaler, MissingValuesFiller

import yfinance as yf
aapl= yf.Ticker(selected_stock)
df = aapl.history(start="2022-04-01", end="2023-04-01", interval="1d")

# data.reset_index(inplace=True)
# data = data[['Date', 'Close']]

idx = pd.date_range(start='2022-04-01', end='2023-04-01')
df.index = pd.DatetimeIndex(df.index)
df.index = df.index.tz_localize(None) 
# # #df['Date'] = pd.to_datetime(df['Date'])
# # #print(df)
# # #df.set_index='Date'
# # #print(df)
# # df.index = pd.DatetimeIndex(df.index)
# # print(df)
df = df.reindex(idx, method = 'pad')
print(df)

NameError: ignored

In [19]:
series = TimeSeries.from_dataframe(df)

adj_series = series.drop_columns(['Open', 'High', 'Low', 'Volume', 'Dividends', 'Stock Splits'])
training, validation = adj_series.split_after(0.8)

from darts.dataprocessing.transformers import Scaler
transformer = Scaler()
train_transformed = transformer.fit_transform(training)
val_transformed = transformer.transform(validation)
series_transformed = transformer.transform(adj_series)



In [ ]:
my_model = RNNModel(
    model="LSTM",
    n_rnn_layers = 3,
    hidden_dim=4,
    dropout=0,
    batch_size=16,
    n_epochs=10,
    optimizer_kwargs={"lr": 1e-3},
    model_name="Air_RNN",
    log_tensorboard=True,
    random_state=42,
    training_length=20,
    input_chunk_length=3,
    force_reset=True,
    save_checkpoints=True,
)
my_model.fit(train_transformed)

In [32]:
!npm install localtunnel

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
updated 1 package and audited 36 packages in 1.079s

3 packages are looking for funding
  run `npm fund` for details

found 0 vulnerabilities



In [33]:
!streamlit run /content/app.py &>/content/logs.txt &

In [34]:
!npx localtunnel --port 8501

npx: installed 22 in 2.461s
your url is: https://tricky-rats-search.loca.lt
^C
